In [1]:
import os

import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from IPython.display import display
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GroupKFold, KFold, train_test_split
from sklearn.preprocessing import (KBinsDiscretizer, OrdinalEncoder,
                                   RobustScaler, normalize, MinMaxScaler)
from sklearn import model_selection


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
df = pd.read_csv("../data/train_5folds.csv")
test_df = pd.read_csv("../data/test.csv")

In [4]:

from pykalman import KalmanFilter
def Kalman1D(observations,damping=1):
    # To return the smoothed time series data
    observation_covariance = damping
    initial_value_guess = observations[0]
    transition_matrix = 1
    transition_covariance = 0.1
    initial_value_guess
    kf = KalmanFilter(
            initial_state_mean=initial_value_guess,
            initial_state_covariance=observation_covariance,
            observation_covariance=observation_covariance,
            transition_covariance=transition_covariance,
            transition_matrices=transition_matrix
        )
    pred_state, state_cov = kf.smooth(observations)
    return pred_state

def add_features(df, transform="robust"):
    df["u_in_log1p"] = np.log1p(df["u_in"])
    df["u_in_kalman"] = Kalman1D(df["u_in"])
    
    df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta'] = df['time_delta'] * df['u_in']
    df['area'] = df.groupby('breath_id')['delta'].cumsum()

    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] / df['count']
    
    df = df.drop(['count','one'], axis=1)

    for i in range(1, 8):    
        df[f'u_in_lag{i}'] = df.groupby('breath_id')['u_in'].shift(i)
        df[f'u_out_lag{i}'] = df.groupby('breath_id')['u_out'].shift(i)
        df[f'u_in_lag_back{i}'] = df.groupby('breath_id')['u_in'].shift(-i)
        df[f'u_out_lag_back{i}'] = df.groupby('breath_id')['u_out'].shift(-i)

    df = df.fillna(0)

    for i in range(1, 8):    
        df[f'u_in_diff{i}'] = df['u_in'] - df[f'u_in_lag{i}']

    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    

    r_map = {5: 0, 20: 1, 50: 2}
    c_map = {10: 0, 20: 1, 50: 2}
    rc_map = {'20_50': 0, '20_20': 1, '50_20': 2, '50_50': 3, '5_50': 4, '5_20': 5, '50_10': 6, '20_10': 7, '5_10': 8}
    df['RC'] = df['R'].astype(str) + '_' + df['C'].astype(str)
    
    df['R'] = df['R'].map(r_map)
    df['C'] = df['C'].map(c_map)
    
    df['RC'] = df['RC'].map(rc_map)
    return df


In [5]:
train_df = add_features(df)
test_df = add_features(test_df)

In [7]:
train_df.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,u_in_kalman,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag5,u_out_lag5,u_in_lag_back5,u_out_lag_back5,u_in_lag6,u_out_lag6,u_in_lag_back6,u_out_lag_back6,u_in_lag7,u_out_lag7,u_in_lag_back7,u_out_lag_back7,u_in_diff1,u_in_diff2,u_in_diff3,u_in_diff4,u_in_diff5,u_in_diff6,u_in_diff7,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_in__diffmax,breath_id__u_in__diffmean,RC
0,1,1,1,2,0.000000,0.083334,0,5.837492,4,0.080043,13.272894,0.000000,0.000000,0.000000,0.0,0.0,0.083334,0.083334,0.000000,0.0,18.383041,0.0,0.000000,0.0,22.509278,0.0,0.000000,0.0,22.808822,0.0,0.000000,0.0,25.355850,0.0,0.0,0.0,27.259866,0.0,0.0,0.0,27.127486,0.0,0.0,0.0,26.807732,0.0,0.083334,0.083334,0.083334,0.083334,0.083334,0.083334,0.083334,28.313036,1,28.229702,10.062673,0
1,2,1,1,2,0.033652,18.383041,0,5.907794,4,2.964399,15.910805,0.033652,0.618632,0.618632,0.0,0.0,18.466375,9.233188,0.083334,0.0,22.509278,0.0,0.000000,0.0,22.808822,0.0,0.000000,0.0,25.355850,0.0,0.000000,0.0,27.259866,0.0,0.0,0.0,27.127486,0.0,0.0,0.0,26.807732,0.0,0.0,0.0,27.864715,0.0,18.299707,18.383041,18.383041,18.383041,18.383041,18.383041,18.383041,28.313036,1,9.929994,-8.237035,0
2,3,1,1,2,0.067514,22.509278,0,7.876254,4,3.157395,18.301494,0.033862,0.762212,1.380843,0.0,0.0,40.975653,13.658551,18.383041,0.0,22.808822,0.0,0.083334,0.0,25.355850,0.0,0.000000,0.0,27.259866,0.0,0.000000,0.0,27.127486,0.0,0.0,0.0,26.807732,0.0,0.0,0.0,27.864715,0.0,0.0,0.0,28.313036,0.0,4.126236,22.425944,22.509278,22.509278,22.509278,22.509278,22.509278,28.313036,1,5.803758,-12.363271,0
3,4,1,1,2,0.101542,22.808822,0,11.742872,4,3.170056,20.271404,0.034028,0.776134,2.156978,0.0,0.0,63.784476,15.946119,22.509278,0.0,25.355850,0.0,18.383041,0.0,27.259866,0.0,0.083334,0.0,27.127486,0.0,0.000000,0.0,26.807732,0.0,0.0,0.0,27.864715,0.0,0.0,0.0,28.313036,0.0,0.0,0.0,26.866758,0.0,0.299544,4.425781,22.725488,22.808822,22.808822,22.808822,22.808822,28.313036,1,5.504214,-12.662816,0
4,5,1,1,2,0.135756,25.355850,0,12.234987,4,3.271690,21.987572,0.034213,0.867507,3.024485,0.0,0.0,89.140326,17.828065,22.808822,0.0,27.259866,0.0,22.509278,0.0,27.127486,0.0,18.383041,0.0,26.807732,0.0,0.083334,0.0,27.864715,0.0,0.0,0.0,28.313036,0.0,0.0,0.0,26.866758,0.0,0.0,0.0,26.762803,0.0,2.547028,2.846573,6.972809,25.272516,25.355850,25.355850,25.355850,28.313036,1,2.957185,-15.209844,0


In [8]:
train_df.describe()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,u_in_kalman,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag5,u_out_lag5,u_in_lag_back5,u_out_lag_back5,u_in_lag6,u_out_lag6,u_in_lag_back6,u_out_lag_back6,u_in_lag7,u_out_lag7,u_in_lag_back7,u_out_lag_back7,u_in_diff1,u_in_diff2,u_in_diff3,u_in_diff4,u_in_diff5,u_in_diff6,u_in_diff7,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_in__diffmax,breath_id__u_in__diffmean,RC
count,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6036000.0,6.036000e+06,6.036000e+06,6.036000e+06
mean,3.018000e+06,6.283886e+04,1.069795e+00,9.547647e-01,1.307225e+00,7.321615e+00,6.204493e-01,1.122041e+01,2.000000e+00,1.452719e+00,7.321613e+00,3.268504e-02,2.366825e-01,1.298736e+01,1.824144e+00,1.123002e+00,4.062300e+02,1.256786e+01,7.262764e+00,6.079493e-01,7.147731e+00,6.204493e-01,7.203947e+00,5.954493e-01,6.797571e+00,6.204493e-01,7.145212e+00,5.829493e-01,6.508529e+00,6.204493e-01,7.086514e+00,5.704493e-01,6.297275e+00,6.204493e-01,7.027904e+00,5.579493e-01,6.062309e+00,6.204493e-01,6.969370e+00,5.454493e-01,5.797680e+00,6.204493e-01,6.910958e+00,5.329493e-01,5.543617e+00,6.204493e-01,5.885066e-02,1.176676e-01,1.764026e-01,2.351004e-01,2.937106e-01,3.522451e-01,4.106564e-01,3.602008e+01,1.0,2.869846e+01,1.669705e-17,4.145951e+00
std,1.742443e+06,3.633526e+04,8.508248e-01,8.345633e-01,7.659778e-01,1.343470e+01,4.852752e-01,8.109703e+00,1.414214e+00,1.095093e+00,1.081787e+01,3.876106e-03,4.347354e-01,1.334446e+01,2.809860e+00,9.555894e-01,4.141439e+02,1.443182e+01,1.345586e+01,4.882079e-01,1.314116e+01,4.852752e-01,1.347672e+01,4.908049e-01,1.242073e+01,4.852752e-01,1.349727e+01,4.930714e-01,1.193367e+01,4.852752e-01,1.351754e+01,4.950121e-01,1.165354e+01,4.852752e-01,1.353750e+01,4.966306e-01,1.133376e+01,4.852752e-01,1.355717e+01,4.979301e-01,1.090741e+01,4.852752e-01,1.357653e+01,4.989132e-01,1.052839e+01,4.852752e-01,7.961707e+00,1.072019e+01,1.064721e+01,1.057946e+01,1.192110e+01,1.310265e+01,1.365175e+01,3.140594e+01,0.0,3.045699e+01,1.225743e+01,2.514575e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.895744e+00,0.000000e+00,0.000000e+00,5.154815e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02,0.000000e+00,1.0,0.000000e+00,-9.602704e+01,0.000000e+00
25%,1.509001e+06,3.137700e+04,0.000000e+00,0.000000e+00,6.428995e-01,3.936623e-01,0.000000e+00,6.329607e+00,1.000000e+00,3.319350e-01,1.750569e+00,3.188229e-02,4.090800e-03,4.284722e+00,0.000000e+00,0.000000e+00,1.349007e+02,3.908123e+00,8.853319e-03,0.000000e+00,1.241114e-01,0.000000e+00,0.000000e+00,0.000000

In [9]:
train_df["u_in_round2"] = np.round(train_df["u_in"], 2)
test_df["u_in_round2"] = np.round(test_df["u_in"], 2)


In [10]:
train_df["kfold"] = -1
y = train_df["pressure"].values

kf = model_selection.GroupKFold(n_splits=10)
# kf = model_selection.StratifiedGroupKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=train_df, y=y, groups=train_df.breath_id.values)):
    print(f)
    train_df.loc[v_, "kfold"] = f

0
1
2
3
4
5
6
7
8
9


In [11]:
train_df.to_csv("../data/train_10folds_nb14.csv", index=False)
test_df.to_csv("../data/test_nb14.csv", index=False)

In [15]:
not_used = ["id", "breath_id", "R", "C", "RC", "pressure", "kfold", "time_step"]
used_columns = list(set(train_df.columns) - set(not_used))
norm_feature = sorted([c for c in sorted(used_columns) if "u_out" not in c])

In [16]:
norm_feature

['area',
 'breath_id__u_in__diffmax',
 'breath_id__u_in__diffmean',
 'breath_id__u_in__max',
 'cross',
 'cross2',
 'delta',
 'time_delta',
 'u_in',
 'u_in_cummean',
 'u_in_cumsum',
 'u_in_diff1',
 'u_in_diff2',
 'u_in_diff3',
 'u_in_diff4',
 'u_in_diff5',
 'u_in_diff6',
 'u_in_diff7',
 'u_in_kalman',
 'u_in_lag1',
 'u_in_lag2',
 'u_in_lag3',
 'u_in_lag4',
 'u_in_lag5',
 'u_in_lag6',
 'u_in_lag7',
 'u_in_lag_back1',
 'u_in_lag_back2',
 'u_in_lag_back3',
 'u_in_lag_back4',
 'u_in_lag_back5',
 'u_in_lag_back6',
 'u_in_lag_back7',
 'u_in_log1p',
 'u_in_round2']

In [17]:
RS = RobustScaler()
train_df[norm_feature] = RS.fit_transform(train_df[norm_feature])
test_df[norm_feature] = RS.fit_transform(test_df[norm_feature])



In [18]:
train_df.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,u_in_kalman,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag5,u_out_lag5,u_in_lag_back5,u_out_lag_back5,u_in_lag6,u_out_lag6,u_in_lag_back6,u_out_lag_back6,u_in_lag7,u_out_lag7,u_in_lag_back7,u_out_lag_back7,u_in_diff1,u_in_diff2,u_in_diff3,u_in_diff4,u_in_diff5,u_in_diff6,u_in_diff7,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_in__diffmax,breath_id__u_in__diffmean,RC,u_in_round2
0,1,1,1,2,0.000000,-0.937384,0,5.837492,9,-1.100643,1.696041,-15.475938,-0.860663,-0.734997,0.0,-0.665541,-0.725228,-0.648438,-0.864121,0.0,2.890207,0.0,-0.842527,0.0,3.664026,0.0,-0.819258,0.0,3.743915,0.0,-0.794152,0.0,4.277824,0.0,-0.76915,0.0,4.683263,0.0,-0.738446,0.0,4.686223,0.0,-0.703447,0.0,4.653368,0.0,0.471409,0.217993,0.121028,0.068315,0.023523,0.003111,-0.016488,0.112208,1,0.326941,1.273864,0,-0.938998
1,2,1,1,2,0.033652,3.049278,0,5.907794,9,0.878825,2.202961,0.103311,2.863270,-0.683536,0.0,-0.665541,-0.676829,0.162935,-0.847360,0.0,3.739712,0.0,-0.842527,0.0,3.724218,0.0,-0.819258,0.0,4.256189,0.0,-0.794152,0.0,4.661047,0.0,-0.76915,0.0,4.656595,0.0,-0.738446,0.0,4.621744,0.0,-0.703447,0.0,4.866757,0.0,113.467773,58.762670,38.091005,26.306250,18.058084,12.987738,10.056416,0.112208,1,-0.178436,-1.450205,0,3.047930
2,3,1,1,2,0.067514,3.948195,0,7.876254,9,1.011274,2.662374,0.200442,3.727568,-0.620131,0.0,-0.665541,-0.617568,0.555359,2.833140,0.0,3.801381,0.0,-0.825784,0.0,4.236036,0.0,-0.819258,0.0,4.639136,0.0,-0.794152,0.0,4.634403,0.0,-0.76915,0.0,4.592180,0.0,-0.738446,0.0,4.834887,0.0,-0.703447,0.0,4.957266,0.0,25.549576,71.696779,46.652513,32.222406,22.124535,15.915525,12.327665,0.112208,1,-0.292389,-2.064430,0,3.947712
3,4,1,1,2,0.101542,4.013452,0,11.742872,9,1.019963,3.040926,0.277152,3.811377,-0.555568,0.0,-0.665541,-0.557517,0.758212,3.663022,0.0,4.325760,0.0,2.850764,0.0,4.618643,0.0,-0.802501,0.0,4.612511,0.0,-0.794152,0.0,4.570046,0.0,-0.76915,0.0,4.805110,0.0,-0.738446,0.0,4.925291,0.0,-0.703447,0.0,4.665284,0.0,1.812565,14.110409,47.101127,32.651890,22.419739,16.128068,12.492546,0.112208,1,-0.300662,-2.109020,0,4.013072
4,5,1,1,2,0.135756,4.568332,0,12.234987,9,1.089713,3.370717,0.363024,4.361409,-0.483405,0.0,-0.665541,-0.490761,0.925096,3.723268,0.0,4.717757,0.0,3.679756,0.0,4.592042,0.0,2.877119,0.0,4.548200,0.0,-0.777379,0.0,4.782785,0.0,-0.76915,0.0,4.895425,0.0,-0.738446,0.0,4.633647,0.0,-0.703447,0.0,4.644297,0.0,15.753730,9.058184,14.415967,36.184309,24.929863,17.935321,13.894534,0.112208,1,-0.371002,-2.488167,0,4.568627


In [ ]:
train_df.to_csv("../data/train_10folds_nb14_robust.csv", index=False)
test_df.to_csv("../data/test_nb14.csv", index=False)